To install ExTEMPO from GitHub, run the following line from terminal window or command prompt. (Only run this once)

"pip install git+https://github.com/BRajkumar041992/ExTEMPO.git"

In [ ]:
import ExTEMPO
import os
import numpy as np
import pandas as pd

In [ ]:
#User Inputs
#--------------------------------#

sample_file = "/stellar_params.csv" #Input full path to the sample stars and their stellar parameters.

aadg3_path = "/AADG3-v3.0.2/bin/AADG3"  
# Input full path to the AADG3 executable.

base_path = "/Results" 
#Input full path to directory where outputs will be stored.

Star_index = 14 # Index of the star in the sample file. (14 = Sun)
N_rel = 100 # Number of Monte Carlo samples/realisations.

#Set inputs
ints = np.arange(120, 6001, 10)  # Integration times in seconds. Uses integrations times from 2 minutes to 1 hour, in steps of 10 seconds. Smaller steps or longer times will increase the runtime.
NU_GRID = np.linspace(0, 487200, 500000)  # Define frequency grid for the PSD
sample = ExTEMPO.load_sample_subset(sample_file)
star_id = f"{sample.loc[Star_index]['primary_name']}" # Star ID from the sample
star_folder = f"{base_path}/{star_id}"

#Parameters and Settings for AADG3
#--------------------------------#

length_sec, cadence_sec = ExTEMPO.time_series_properties(NU_GRID)  # Calculates the time series length and cadence from the frequency grid
T_fixed = length_sec  # Total duration of observations in seconds
cadence = 1         # Cadence in seconds.
n_cadences = int(T_fixed / cadence)
granulation_on = False # Toggle granulation signal True (on) or False (off).


config_overrides = {
    "cadence": cadence,                # Cadence of timeseries second (e.g., '1d0' for 1 second).
    "n_cadences": n_cadences,          # Total number of cadences in the time series (72 days at 1 second cadence).
    "rho": 0.45,                          # Correlation coefficient for the driving term (0 for purely uncorrelated, 1 for pure correlated).
    "n_relax": 8640,                  # Number of cadences over which the stochastic driving term is allowed to build up from zero.
    "n_fine": 50,                      # Number of sub-cadences, the cadence is divided into when the driving term is generated.
    "inclination": 90,                 # Inclination angle of the stellar rotation axis (degrees): 90 for edge-on, 0 for pole-on.
    "p1": 1.35,                        # Visibility factor for l=1 modes, for HARPS (535 nm): Handberg and Campante (2011).
    "p2": 1.02,                        # Visibility factor for l=2 modes, for HARPS (535 nm): Handberg and Campante (2011).
    "p3": 0.48,                        # Visibility factor for l=3 modes, for HARPS (535 nm): Handberg and Campante (2011).
    "add_granulation": ".true" if granulation_on else ".false"                  
}


star_id


In [ ]:
ExTEMPO.monte_carlo(index=Star_index, sample_file=sample_file, N=N_rel, base_path=base_path)
#Run Monte Carlo sampling to generate stellar parameters realisations.

ExTEMPO.run_all_realisations_parallel(star_id, base_path=base_path, NU_GRID=NU_GRID, max_workers=4, **config_overrides)
#Generates AADG3 inputfiles for all realisations.

ExTEMPO.run_all_aadg3_parallel(star_id, base_path=base_path, aadg3_path=aadg3_path, N=N_rel, max_workers=4)
#Runs AADG3 for all realisations in parallel.

ExTEMPO.analyze_all_realisations(base_path, star_id, ints, T_fixed, max_realisations=N_rel)
#Analyzes all realisations to compute the RMS vs Integration time curves

In [ ]:
#Plot the results of the analysis
#---------------------------------------------------#
 
results = ExTEMPO.plot_sim(
    sample=sample,
    star_name=f"{star_id}",         # Input desired star name from the sample
    mc_index=3,                    # Input desired Monte Carlo realisation index
    base_path=base_path,
    generate_oscillation=True,      #Toggle to generate PSD of P-modes
    generate_linewidth=True,        #Toggle to generate linewidth vs frequency plot
    generate_amplitude=True,        #Toggle to generate amplitude by degree plot
    generate_echelle=True,          #Toggle to generate echelle diagram
    generate_time_series=True,      #Toggle to generate synthetic time series plot
    generate_from_npz=True          #Toggle to generate residual amplitude plot
)

In [ ]:
star_f=star_id.replace(' ', '_')
star_folder1 = os.path.join(base_path, star_f)
npz_path = os.path.join(star_folder1, os.path.basename(star_folder.rstrip("/")) + ".npz")

#User Inputs for RMS estimation
#---------------------------------------------------#
exposure_time_min = 5.0     # Set desired exposure time in minutes for P-mode RMS estimation    
target_rms = 0.1            # Set target RMS value in m/s for time estimation

In [ ]:
# Get P-mode RMS amplitude for a specific exposure time
ExTEMPO.get_rms_for_time(npz_path, exposure_time_min=5.0)


In [ ]:
# Get exposure times to achieve a specific target RMS value
#---------------------------------------------------#
min_time, time_err = ExTEMPO.get_time_for_rms(npz_path, target_rms=target_rms,figsize=(10, 8))
min_time, time_err
